In [3]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from pyproj import Geod

# Caminho do arquivo HDF5
arquivo = r'Z:\vol_250km_13steps.vol\2025\07\08\vol_250km_13steps.vol_sbbe_20250708171006_00.h5'

# Parâmetros fixos
range_step = 240  # metros por bin
geod = Geod(ellps='WGS84')

with h5py.File(arquivo, 'r') as f:
    # === LER DADOS DE REFLETIVIDADE ===
    dataset = f['scan5/moment_0'][:]
    refletividade = dataset.astype(np.float32) * 0.5 - 32

    # === LER LATITUDE E LONGITUDE DO RADAR DE /where ===
    lat_radar = f['where'].attrs['lat']
    lon_radar = f['where'].attrs['lon']

    # === CRIAR GRELHA POLAR ===
    azimutes = np.deg2rad(np.linspace(0, 360, dataset.shape[0], endpoint=False))
    alcances = np.arange(dataset.shape[1]) * range_step
    AZ, R = np.meshgrid(azimutes, alcances, indexing='ij')
    x = R * np.sin(AZ)
    y = R * np.cos(AZ)

# === MASCARAR VALORES INVÁLIDOS ===
refletividade = np.ma.masked_where(refletividade <= -32, refletividade)

# === CONVERTER (x, y) PARA LAT/LON USANDO pyproj ===
x_flat = x.ravel()
y_flat = y.ravel()
az_flat = np.rad2deg(np.arctan2(x_flat, y_flat))
dist_flat = np.sqrt(x_flat**2 + y_flat**2)
n = len(x_flat)

lat_radar_array = np.full(n, lat_radar)
lon_radar_array = np.full(n, lon_radar)

lon_flat, lat_flat, _ = geod.fwd(lon_radar_array, lat_radar_array, az_flat, dist_flat)
lon = lon_flat.reshape(x.shape)
lat = lat_flat.reshape(y.shape)

# === PLOTAR REFLETIVIDADE SEM EIXOS OU LEGENDA ===
fig, ax = plt.subplots(figsize=(10, 10))
ax.set_position([0, 0, 1, 1])
ax.axis('off')

plt.pcolormesh(lon, lat, refletividade, cmap='turbo', shading='auto', vmin=-32, vmax=64)

# SALVAR COMO IMAGEM PNG (com fundo transparente)
plt.savefig('refletividade_georef.png', dpi=300, bbox_inches='tight', pad_inches=0, transparent=True)
plt.close()


C:\Users\cleber.santos\AppData\Local\Temp\ipykernel_15736\3764862199.py:51: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  plt.pcolormesh(lon, lat, refletividade, cmap='turbo', shading='auto', vmin=-32, vmax=64)
